In [2]:
import pandas as pd
import numpy as np

In [46]:
data_correct=pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-zakorko/Courses/KC_case_data .csv')

In [47]:
data_correct['date'] = pd.to_datetime(data_correct['date'])

In [48]:
data_correct=data_correct.rename(columns={'utm_source':'urm_source'})

In [49]:
data_correct

,date,event,purchase_sum,os_name,device_id,gender,city,urm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [50]:
data_correct.query("date>='2020-02-01' and date<='2020-02-29'").agg({'device_id':'nunique'})

device_id    75032
dtype: int64

In [51]:
data_correct.query("date>='2020-01-01' and date<='2020-01-31' and event=='app_install'").agg({'device_id':'count'})

device_id    80297
dtype: int64

In [52]:

# Фильтрация данных по событиям установки и покупки
install_events = data_correct[data_correct['event'] == 'app_install']
purchase_events = data_correct[data_correct['event'] == 'purchase']

# Определение даты установки для каждого пользователя
install_dates = install_events.groupby('device_id')['date'].min().reset_index()
install_dates.rename(columns={'date': 'install_date'}, inplace=True)

# Определение даты первой покупки для каждого пользователя
first_purchase_dates = purchase_events.groupby('device_id')['date'].min().reset_index()
first_purchase_dates.rename(columns={'date': 'first_purchase_date'}, inplace=True)

# Объединение данных по установке и покупке
conversion_data = pd.merge(install_dates, first_purchase_dates, on='device_id', how='left')

# Расчет разницы в днях между установкой и первой покупкой
conversion_data['days_to_purchase'] = (conversion_data['first_purchase_date'] - conversion_data['install_date']).dt.days

# Определение конверсии в покупку в течение 7 дней
conversion_data['converted_within_7_days'] = conversion_data['days_to_purchase'].between(0, 7)

# Расчет конверсии для каждой когорты по дню установки
conversion_rates = conversion_data.groupby(conversion_data['install_date'].dt.date)['converted_within_7_days'].mean().reset_index()

# Определение когорты с наибольшей конверсией
highest_conversion_cohort = conversion_rates.loc[conversion_rates['converted_within_7_days'].idxmax()]

highest_conversion_cohort


install_date               2020-01-01
converted_within_7_days      0.393406
Name: 0, dtype: object

In [53]:
data_correct.groupby('urm_source').agg({'device_id':'nunique'})

,device_id
urm_source,
-,52273
facebook_ads,25959
google_ads,38096
instagram_ads,31048
referal,15926
vk_ads,34086
yandex-direct,40712


In [54]:
# Предположим, что событие регистрации обозначается в данных как 'registration'
# Добавим колонку с датой регистрации для каждого пользователя
registration_dates = data_correct[data_correct['event'] == 'registration'].groupby('device_id')['date'].min().reset_index()
registration_dates.rename(columns={'date': 'registration_date'}, inplace=True)

# Добавление информации о дате регистрации к основным данным
data_with_registration = pd.merge(data_correct, registration_dates, on='device_id', how='left')

# Определение статуса регистрации на момент совершения события
data_with_registration['registered'] = data_with_registration['date'] >= data_with_registration['registration_date']

# Определение ключевых этапов воронки и расчет конверсии для зарегистрированных и незарегистрированных пользователей
funnel_steps = ['search', 'add_to_cart', 'begin_checkout', 'registration', 'purchase']

# Инициализация словарей для хранения результатов
conversion_results_registered = {}
conversion_results_unregistered = {}

for i in range(len(funnel_steps)-1):
    prev_step = funnel_steps[i]
    next_step = funnel_steps[i+1]
    
    # Фильтрация данных по этапам для зарегистрированных и незарегистрированных пользователей
    prev_step_data_registered = data_with_registration[(data_with_registration['event'] == prev_step) & (data_with_registration['registered'])]
    next_step_data_registered = data_with_registration[(data_with_registration['event'] == next_step) & (data_with_registration['registered'])]
    
    prev_step_data_unregistered = data_with_registration[(data_with_registration['event'] == prev_step) & (~data_with_registration['registered'])]
    next_step_data_unregistered = data_with_registration[(data_with_registration['event'] == next_step) & (~data_with_registration['registered'])]
    
    # Расчет конверсии
    conversion_rate_registered = len(next_step_data_registered['device_id'].unique()) / len(prev_step_data_registered['device_id'].unique()) if len(prev_step_data_registered['device_id'].unique()) > 0 else 0
    conversion_rate_unregistered = len(next_step_data_unregistered['device_id'].unique()) / len(prev_step_data_unregistered['device_id'].unique()) if len(prev_step_data_unregistered['device_id'].unique()) > 0 else 0
    
    conversion_results_registered[(prev_step, next_step)] = conversion_rate_registered
    conversion_results_unregistered[(prev_step, next_step)] = conversion_rate_unregistered

# Определение этапа с наименьшей конверсией для зарегистрированных пользователей
lowest_conversion_step_registered = min(conversion_results_registered, key=conversion_results_registered.get)

lowest_conversion_step_registered


('search', 'add_to_cart')

In [68]:
# Расчет конверсии, учитывая только уникальных пользователей
# Определение уникальных пользователей, впервые открывших приложение, по каналам
unique_first_app_start = first_app_start.groupby('urm_source')['device_id'].nunique().reset_index()

# Определение уникальных пользователей, совершивших первую покупку, по каналам
unique_first_purchase = pd.merge(first_app_start, first_purchase, on='device_id', how='inner').groupby('urm_source')['device_id'].nunique().reset_index()

# Расчет конверсии для каждого канала
conversion_by_channel_unique = pd.merge(unique_first_app_start, unique_first_purchase, on='urm_source', how='left', suffixes=('_app_start', '_purchase'))
conversion_by_channel_unique['conversion_rate'] = conversion_by_channel_unique['device_id_purchase'] / conversion_by_channel_unique['device_id_app_start']

# Исключаем канал с обозначением '-' из анализа и ищем канал с наименьшей конверсией
lowest_conversion_channel_corrected = conversion_rate_by_channel[conversion_rate_by_channel['urm_source'] != '-'].iloc[0]

lowest_conversion_channel_corrected['urm_source']



'google_ads'

In [72]:
# Определение первой покупки каждого пользователя и суммы этой покупки
first_purchase_sum = purchase_events.groupby('device_id').first().reset_index()[['device_id', 'purchase_sum']]

# Сопоставление каждой первой покупки с каналом, из которого пришел пользователь
first_purchase_with_channel = pd.merge(first_purchase_sum, first_app_start[['device_id', 'urm_source']], on='device_id', how='left')

# Расчет медианного первого чека для каждого канала
median_first_check_by_channel = first_purchase_with_channel.groupby('urm_source')['purchase_sum'].median().reset_index()

# Определение канала с наивысшим медианным первым чеком
highest_median_first_check_channel = median_first_check_by_channel.sort_values(by='purchase_sum', ascending=False).iloc[0]

highest_median_first_check_channel['urm_source']


'referal'

In [79]:
# Затраты на рекламу по каналам
ad_costs = {
    'yandex': 10491707,
    'google': 10534878,
    'facebook': 8590498,
    'instagram': 8561626,
    'vk': 9553531,
}

# Суммарный доход от первых покупок по каналам
total_revenue_by_channel = first_purchase_with_channel.groupby('urm_source')['purchase_sum'].sum().reset_index()

# Преобразование названий каналов для соответствия ключам словаря затрат
total_revenue_by_channel['urm_source'] = total_revenue_by_channel['urm_source'].replace({
    'yandex_direct': 'yandex',
    'google_ads': 'google',
    'facebook_ads': 'facebook',
    'instagram_ads': 'instagram',
    'vk_ads': 'vk',
    'referal': 'referral_program'
})

# Расчет ROMI для каждого рекламного канала
romi_by_channel = []
for channel, cost in ad_costs.items():
    revenue = total_revenue_by_channel.loc[total_revenue_by_channel['urm_source'] == channel, 'purchase_sum'].values[0] if channel in total_revenue_by_channel['urm_source'].values else 0
    romi = ((revenue - cost) / cost) * 100 if cost > 0 else 0
    romi_by_channel.append({'channel': channel, 'ROMI': romi})

# Определение канала с самым высоким ROMI
romi_df = pd.DataFrame(romi_by_channel)
highest_romi_channel = romi_df.sort_values(by='ROMI', ascending=False).iloc[0]['channel']

# Преобразование названий каналов обратно для вывода ответа
channel_names_map = {
    'yandex': 'Яндекс',
    'google': 'Гугл',
    'facebook': 'Фейсбук',
    'instagram': 'Инстаграм',
    'vk': 'ВК',
}
highest_romi_channel_name = channel_names_map[highest_romi_channel]

highest_romi_channel_name


'ВК'